# Encryptee – C# Interactive Notebook

Tento notebook slouží jako interaktivní průvodce projektem **Encryptee**. Cílem je:

1. Srozumitelně ukázat architekturu (Strategy, Observer, Decorator).
2. Vysvětlit všechny důležité proměnné a komponenty.
3. Umožnit okamžité spuštění ukázek šifrování/dešifrování bez konzolového vstupu.

Notebook používá jádro **.NET (C#)** v prostředí Jupyter nebo VS Code (.NET Interactive/Polyglot Notebooks).


## 1) Základní using direktivy

Načteme jmenné prostory, které využíváme v celém projektu.

In [ ]:
using System;
using System.Linq;
using System.Security.Cryptography;
using System.Text;
using System.Text.Json;


## 2) Logger (Decorator)

Logger je navržen pomocí **Decorator** patternu. Základní `ConsoleLogger` lze obalit dekorátory jako `JsonDecorator` a `TimestampDecorator`, aniž bychom měnili původní kód loggeru. Pole **Inner** v dekorátoru odkazuje na další logger v řetězci.

In [ ]:
// Rozhraní loggeru
public interface ILogger { void Log(string line); }

// Základní logger – vypisuje do konzole
public sealed class ConsoleLogger : ILogger
{
    public void Log(string line) => Console.WriteLine(line);
}

// Abstraktní dekorátor – drží odkaz na vnořený logger a deleguje volání
public abstract class LoggerDecorator : ILogger
{
    protected readonly ILogger Inner;
    protected LoggerDecorator(ILogger inner) => Inner = inner;
    public virtual void Log(string line) => Inner.Log(line);
}

// Dekorátor: časové razítko
public sealed class TimestampDecorator : LoggerDecorator
{
    public TimestampDecorator(ILogger inner) : base(inner) { }
    public override void Log(string line) => base.Log($"{DateTime.Now:yyyy-MM-dd HH:mm:ss.fff} | {line}");
}

// Dekorátor: JSON formát
public sealed class JsonDecorator : LoggerDecorator
{
    private readonly string _source;
    private readonly string _level;
    public JsonDecorator(ILogger inner, string source = "encryptee", string level = "INFO") : base(inner)
    {
        _source = source; _level = level;
    }
    public override void Log(string line)
    {
        var payload = new { ts = DateTime.UtcNow, level = _level, source = _source, message = line };
        base.Log(JsonSerializer.Serialize(payload));
    }
}


## 3) Observer (události)

Observer odděluje **události běhu** od jejich zobrazení. Engine pouze oznamuje, že začal, dokončil, nebo nastala chyba. `ConsoleCryptoObserver` přijímá tyto události a přeposílá je loggeru.

In [ ]:
// Události šifrovací operace
public interface ICryptoObserver
{
    void OnStart(string mode, string algo);
    void OnComplete(string outputPreviewBase64, TimeSpan elapsed);
    void OnError(string reason);
}

// Konzolový pozorovatel využívající logger pipeline
public sealed class ConsoleCryptoObserver : ICryptoObserver
{
    private readonly ILogger _log;
    public ConsoleCryptoObserver(ILogger log) => _log = log;

    public void OnStart(string mode, string algo)
        => _log.Log($"START | mode={mode} | algo={algo}");

    public void OnComplete(string outputPreviewBase64, TimeSpan elapsed)
        => _log.Log($"DONE  | elapsed={elapsed} | preview(base64)={outputPreviewBase64}");

    public void OnError(string reason)
        => _log.Log($"ERROR | {reason}");
}


## 4) Strategy (šifrovací algoritmy)

Každý algoritmus implementuje jednotné rozhraní `IEncryptStrategy`. Engine volá pouze metodu `Process`, aniž by znal detaily implementace.

In [ ]:
public interface IEncryptStrategy
{
    // mode = "enc" nebo "dec"
    string Process(string input, string passwordOrKey, string mode);
}

// Caesar – posun ASCII písmen A-Z/a-z, ostatní znaky beze změny
public sealed class CaesarStrategy : IEncryptStrategy
{
    private readonly int _shift;
    public CaesarStrategy(int shift) => _shift = ((shift % 26) + 26) % 26;

    public string Process(string input, string key, string mode)
    {
        int s = _shift * (mode == "dec" ? -1 : 1);
        return new string(input.Select(Shift).ToArray());

        char Shift(char c)
        {
            if (c >= 'a' && c <= 'z') return (char)('a' + (c - 'a' + s + 26) % 26);
            if (c >= 'A' && c <= 'Z') return (char)('A' + (c - 'A' + s + 26) % 26);
            return c;
        }
    }
}

// XOR – didaktická ukázka, výstup Base64
public sealed class XorStrategy : IEncryptStrategy
{
    public string Process(string input, string key, string mode)
    {
        if (string.IsNullOrEmpty(key)) throw new ArgumentException("Key cannot be empty.");

        if (mode == "enc")
        {
            var data = Encoding.UTF8.GetBytes(input);
            var res = Xor(data, Encoding.UTF8.GetBytes(key));
            return Convert.ToBase64String(res);
        }
        else if (mode == "dec")
        {
            var data = Convert.FromBase64String(input);
            var res = Xor(data, Encoding.UTF8.GetBytes(key));
            return Encoding.UTF8.GetString(res);
        }
        else throw new ArgumentException("mode must be 'enc' or 'dec'");
    }

    private static byte[] Xor(byte[] data, byte[] key)
    {
        var res = new byte[data.Length];
        for (int i = 0; i < data.Length; i++)
            res[i] = (byte)(data[i] ^ key[i % key.Length]);
        return res;
    }
}

// AES-CBC s PBKDF2 (Rfc2898DeriveBytes) a formátem base64(salt|iv|cipher)
public sealed class AesPbkdf2Strategy : IEncryptStrategy
{
    private const int SaltSize = 16;
    private const int IvSize = 16;
    private const int KeySizeBytes = 32;
    private const int Iterations = 100_000;

    public string Process(string input, string password, string mode)
    {
        if (string.IsNullOrEmpty(password)) throw new ArgumentException("Password cannot be empty.");

        if (mode == "enc")
        {
            var salt = RandomBytes(SaltSize);
            using var keyDerive = new Rfc2898DeriveBytes(password, salt, Iterations, HashAlgorithmName.SHA256);
            var key = keyDerive.GetBytes(KeySizeBytes);

            using var aes = Aes.Create();
            aes.Mode = CipherMode.CBC;
            aes.Padding = PaddingMode.PKCS7;
            aes.Key = key;
            aes.GenerateIV();

            using var enc = aes.CreateEncryptor();
            var plaintext = Encoding.UTF8.GetBytes(input);
            var cipher = enc.TransformFinalBlock(plaintext, 0, plaintext.Length);

            var blob = Combine(salt, aes.IV, cipher);
            return Convert.ToBase64String(blob);
        }
        else if (mode == "dec")
        {
            var blob = Convert.FromBase64String(input);
            if (blob.Length < SaltSize + IvSize + 1) throw new ArgumentException("Invalid AES blob.");
            var salt = blob.Take(SaltSize).ToArray();
            var iv = blob.Skip(SaltSize).Take(IvSize).ToArray();
            var cipher = blob.Skip(SaltSize + IvSize).ToArray();

            using var keyDerive = new Rfc2898DeriveBytes(password, salt, Iterations, HashAlgorithmName.SHA256);
            var key = keyDerive.GetBytes(KeySizeBytes);

            using var aes = Aes.Create();
            aes.Mode = CipherMode.CBC;
            aes.Padding = PaddingMode.PKCS7;
            aes.Key = key;
            aes.IV = iv;

            using var dec = aes.CreateDecryptor();
            var plain = dec.TransformFinalBlock(cipher, 0, cipher.Length);
            return Encoding.UTF8.GetString(plain);
        }
        else throw new ArgumentException("mode must be 'enc' or 'dec'");
    }

    private static byte[] RandomBytes(int n)
    {
        var b = new byte[n];
        RandomNumberGenerator.Fill(b);
        return b;
    }

    private static byte[] Combine(params byte[][] arrays)
    {
        var len = arrays.Sum(a => a.Length);
        var res = new byte[len];
        int pos = 0;
        foreach (var a in arrays) { Buffer.BlockCopy(a, 0, res, pos, a.Length); pos += a.Length; }
        return res;
    }
}


## 5) Engine (orchestrátor)

Engine spojuje strategii a pozorovatele. Neřeší detaily šifrování ani logování. Jen volá `Process` na strategii a ohlašuje události pozorovateli.

In [ ]:
public sealed class EncrypteeEngine
{
    private readonly IEncryptStrategy _strategy;
    private readonly ICryptoObserver _observer;

    public EncrypteeEngine(IEncryptStrategy strategy, ICryptoObserver observer)
    {
        _strategy = strategy;
        _observer = observer;
    }

    public string Run(string mode, string input, string passwordOrKey)
    {
        var sw = System.Diagnostics.Stopwatch.StartNew();
        try
        {
            _observer.OnStart(mode, _strategy.GetType().Name);
            var output = _strategy.Process(input, passwordOrKey, mode);

            var preview = mode == "enc"
                ? output
                : Convert.ToBase64String(Encoding.UTF8.GetBytes(output));

            sw.Stop();
            _observer.OnComplete(preview, sw.Elapsed);
            return output;
        }
        catch (Exception ex)
        {
            sw.Stop();
            _observer.OnError(ex.Message);
            throw;
        }
    }
}


## 6) Pomocné konstrukce pro notebook (bez konzolového vstupu)

V notebooku místo `Console.ReadLine()` použijeme jednoduchý spouštěč `RunDemo`, který přijímá parametry. Vytvoříme také připravený logger pipeline a observer.

In [ ]:
public static class NotebookSetup
{
    public static ILogger BuildLogger(bool asJson)
    {
        ILogger baseLogger = new ConsoleLogger();
        if (asJson)
        {
            return new TimestampDecorator(new JsonDecorator(baseLogger, source: "encryptee"));
        }
        else
        {
            return new TimestampDecorator(baseLogger);
        }
    }

    public static IEncryptStrategy SelectStrategy(string algo, int caesarShift = 3)
    {
        return algo switch
        {
            "caesar" => new CaesarStrategy(caesarShift),
            "xor"    => new XorStrategy(),
            "aes"    => new AesPbkdf2Strategy(),
            _         => new AesPbkdf2Strategy(),
        };
    }
}

public static class NotebookRunner
{
    // Spustí operaci a vrátí výsledný řetězec (Base64 pro enc, UTF-8 pro dec)
    public static string RunDemo(string mode, string algo, string input, string keyOrPass, bool jsonLogs = true, int caesarShift = 3)
    {
        var logger = NotebookSetup.BuildLogger(jsonLogs);
        var observer = new ConsoleCryptoObserver(logger);
        var strategy = NotebookSetup.SelectStrategy(algo, caesarShift);
        var engine = new EncrypteeEngine(strategy, observer);
        return engine.Run(mode, input, keyOrPass);
    }
}


## 7) Ukázky běhu (encrypt/decrypt)

Následující buňky ukazují běžný tok: šifrování a dešifrování pro různé algoritmy. Logy lze přepínat mezi JSON a prostým textem.

In [ ]:
// Caesar – enc/dec (posun 3)
var cEnc = NotebookRunner.RunDemo("enc", "caesar", "Hello World", keyOrPass: "", jsonLogs: false, caesarShift: 3);
var cDec = NotebookRunner.RunDemo("dec", "caesar", cEnc, keyOrPass: "", jsonLogs: false, caesarShift: 3);
Console.WriteLine($"Caesar enc: {cEnc}");
Console.WriteLine($"Caesar dec: {cDec}");


In [ ]:
// XOR – enc/dec (klíč: secret)
var xEnc = NotebookRunner.RunDemo("enc", "xor", "Hello World", keyOrPass: "secret", jsonLogs: true);
var xDec = NotebookRunner.RunDemo("dec", "xor", xEnc, keyOrPass: "secret", jsonLogs: true);
Console.WriteLine($"XOR enc (base64): {xEnc}");
Console.WriteLine($"XOR dec (utf8): {xDec}");


In [ ]:
// AES – enc/dec (heslo: strongpass)
var aEnc = NotebookRunner.RunDemo("enc", "aes", "Hello World", keyOrPass: "strongpass", jsonLogs: true);
var aDec = NotebookRunner.RunDemo("dec", "aes", aEnc, keyOrPass: "strongpass", jsonLogs: true);
Console.WriteLine($"AES enc (base64): {aEnc}");
Console.WriteLine($"AES dec (utf8): {aDec}");


## 8) Vysvětlení datového toku a odpovědností

- Program/Notebook:
  - Sestaví logger pipeline (Console → Json → Timestamp, nebo jen Timestamp).
  - Vytvoří ConsoleCryptoObserver s daným loggerem.
  - Vybere strategii podle parametru (`caesar`, `xor`, `aes`).
  - Spustí engine s parametry `mode`, `input`, `keyOrPass`.

- Engine:
  - Nahlásí start, zavolá strategii, nahlásí dokončení, vrátí výsledek.
  - Případné výjimky nahlásí přes `OnError`.

- Strategy:
  - Implementuje `Process(input, key, mode)`.
  - Caesar: posun ASCII písmen, výstup text.
  - XOR: bajtový XOR, výstup Base64.
  - AES-PBKDF2: bezpečnější varianta, výstup Base64 s `salt|iv|cipher`.

- Observer:
  - Převádí události z engine na logovací zprávy.
  - Nezpracovává citlivá data; loguje pouze meta-informace (start, elapsed, preview, error).

- Logger (Decorator):
  - `ConsoleLogger` je základ.
  - `JsonDecorator` převádí log na JSON.
  - `TimestampDecorator` přidává časové razítko.
